In [1]:
from segmentation.scr.utils.utils import set_seed
from segmentation.scr.utils.metrics import dice_coef
from segmentation.models.smp import get_pretrained_model
from segmentation.config import CFG
import torch
import torch.nn as nn
import numpy as np

from segmentation.scr.train_model.data_loader import *
from segmentation.scr.utils.utils import set_seed
from segmentation.scr.train_model.evaluate_dataset import evaluate_dataset




ImportError: cannot import name 'Unet' from 'segmentation.models.smp.decoder' (c:\Users\123\Desktop\human vasculature\segmentation\models\smp\decoder\__init__.py)

In [2]:
class CFG_3:
    # ============== dice metrics ============
    
    target_size = 1
    in_chans = 3  # 65
    # ============== training CFG =============
    image_size = 512
    input_size = 512
    device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
path_to_model_3_kaag = 'weight/mobnet_3_ch.pth'

#ckpt = torch.load(path_to_model_3_kaag)
#ckpt = ckpt["model"]
#torch.save(ckpt, 'weight/mobnet_3_ch.pth')
model_3_kaag = get_pretrained_model(path_to_model=path_to_model_3_kaag, train_parallel=True, CFG=CFG_3)


In [4]:
path_to_model_3_kaag = 'C:\\Users\\123\\Desktop\\resnet baseline\\state efficientnet-b7_best_model_at_22.pth'

ckpt = torch.load(path_to_model_3_kaag)
ckpt = ckpt["model"]
torch.save(ckpt, 'weight/mobnet_1_5_ch_kaagle.pth')

In [5]:
model_my_pc= get_pretrained_model(path_to_model='weight\mobnet_my_pc.pth', train_parallel=False, CFG=CFG)
model_5_k = get_pretrained_model(path_to_model='weight/mobnet_5_ch_kaagle.pth', train_parallel=True, CFG=CFG)
model_1_5_k = get_pretrained_model(path_to_model='weight/mobnet_1_5_ch_kaagle.pth', train_parallel=True, CFG=CFG)

In [6]:
path_img_dir="data\\train\\kidney_3_sparse\\images"
path_lb_dir="data\\train\\kidney_3_dense\\labels"
img_path, lb_path = create_img_lb_paths(path_img_dir = path_img_dir, path_lb_dir = path_lb_dir)

Общее число изображений с масками сегментации : 501


In [7]:
img_path[500]

'data\\train\\kidney_3_sparse\\images\\0996.tif'

In [8]:
lb_path[500]

'data\\train\\kidney_3_dense\\labels\\0996.tif'

In [9]:
val_x=load_data(img_path ,is_label=False)
print(val_x.shape)
val_y=load_data(lb_path ,is_label=True)
print(val_y.shape)

100%|██████████| 32/32 [00:05<00:00,  5.95it/s]


torch.Size([501, 1706, 1510])


100%|██████████| 32/32 [00:05<00:00,  6.15it/s]

torch.Size([501, 1706, 1510])


In [10]:
set_seed(42)
val_dataset_3=Kaggld_Dataset([val_x],[val_y], arg=False, CFG=CFG_3)
val_loader_3 = DataLoader(val_dataset_3, batch_size=CFG.valid_batch_size, shuffle=False, )

val_dataset_5=Kaggld_Dataset([val_x],[val_y], arg=False, CFG=CFG)
val_loader_5 = DataLoader(val_dataset_5, batch_size=CFG.valid_batch_size, shuffle=False, )

In [32]:
def evaluate_dataset(model, val_loader):
    model.eval()
    timer = tqdm(range(len(val_loader)))
    val_scores = 0
    scores = []
    for i, (x, y) in enumerate(val_loader):
        x = x.cuda().to(torch.float32)
        y = y.cuda().to(torch.float32)
        x = norm_with_clip(x.reshape(-1, *x.shape[2:])).reshape(x.shape)

        with torch.no_grad():
            pred = model(x)

        score = dice_coef(pred.detach(), y)
        scores.append(score.detach().cpu().item())
        val_scores = (val_scores * i + score) / (i + 1)
        timer.set_description(f"eval--> score:{val_scores:.4f}")
        timer.update()
    timer.close()
    return scores, val_scores

In [ ]:
scores, val_scores = evaluate_dataset(model=model_my_pc, val_loader=val_loader_5)

In [11]:
set_seed(42)
scores_3_ch, val_scores_3_ch = evaluate_dataset(model=model_3_kaag, val_loader=val_loader_3)

eval--> score:0.8304: 100%|██████████| 125/125 [00:09<00:00, 13.74it/s]


In [12]:
np.mean(scores_3_ch)

0.8303852319717407

In [13]:
set_seed(42)
scores_5_ch, val_scores_5_ch = evaluate_dataset(model=model_5_k, val_loader=val_loader_5)

eval--> score:0.8228: 100%|██████████| 124/124 [00:07<00:00, 15.62it/s]


In [14]:
set_seed(42)
scores_1_5_ch, val_scores_1_5_ch = evaluate_dataset(model=model_1_5_k, val_loader=val_loader_5)

eval--> score:0.8083: 100%|██████████| 124/124 [00:07<00:00, 16.15it/s]


In [15]:
np.mean(scores_1_5_ch)

0.8083399395428358

In [16]:
scores_my, val_scores_my= evaluate_dataset(model=model_my_pc ,val_loader=val_loader_5)

eval--> score:0.8393: 100%|██████████| 124/124 [00:07<00:00, 15.76it/s]


In [17]:
np.mean(scores_my)

0.8393328303411123

In [31]:
import scipy
scipy.stats.ttest_rel(a=scores_my, b=scores_3_ch[0:124])

TtestResult(statistic=0.6000267680038004, pvalue=0.5495922430215381, df=123)